# Creational Design Patterns in Python

## Introduction
Creational design patterns provide various object creation mechanisms that increase flexibility and reuse of existing code. They help make a system independent of how its objects are created, composed, and represented.

## 1. Singleton Pattern
Ensures a class has only one instance and provides a global point of access to it.

```python
class Singleton:
    _instance = None
    
    def __new__(cls):
        if cls._instance is None:
            cls._instance = super().__new__(cls)
        return cls._instance
    
    def __init__(self):
        self.data = []

# Thread-safe implementation
from threading import Lock

class ThreadSafeSingleton:
    _instance = None
    _lock = Lock()
    
    def __new__(cls):
        with cls._lock:
            if cls._instance is None:
                cls._instance = super().__new__(cls)
        return cls._instance
```

## 2. Factory Method Pattern
Defines an interface for creating objects but lets subclasses decide which classes to instantiate.

```python
from abc import ABC, abstractmethod

# Product interface
class Animal(ABC):
    @abstractmethod
    def speak(self):
        pass

# Concrete products
class Dog(Animal):
    def speak(self):
        return "Woof!"

class Cat(Animal):
    def speak(self):
        return "Meow!"

# Creator interface
class AnimalFactory(ABC):
    @abstractmethod
    def create_animal(self):
        pass

# Concrete creators
class DogFactory(AnimalFactory):
    def create_animal(self):
        return Dog()

class CatFactory(AnimalFactory):
    def create_animal(self):
        return Cat()

# Usage
dog_factory = DogFactory()
dog = dog_factory.create_animal()
print(dog.speak())  # Output: Woof!
```

## 3. Abstract Factory Pattern
Provides an interface for creating families of related or dependent objects without specifying their concrete classes.

```python
from abc import ABC, abstractmethod

# Abstract products
class Button(ABC):
    @abstractmethod
    def paint(self):
        pass

class Checkbox(ABC):
    @abstractmethod
    def paint(self):
        pass

# Concrete products
class WindowsButton(Button):
    def paint(self):
        return "Rendering a Windows button"

class WindowsCheckbox(Checkbox):
    def paint(self):
        return "Rendering a Windows checkbox"

class MacButton(Button):
    def paint(self):
        return "Rendering a Mac button"

class MacCheckbox(Checkbox):
    def paint(self):
        return "Rendering a Mac checkbox"

# Abstract factory
class GUIFactory(ABC):
    @abstractmethod
    def create_button(self) -> Button:
        pass
    
    @abstractmethod
    def create_checkbox(self) -> Checkbox:
        pass

# Concrete factories
class WindowsFactory(GUIFactory):
    def create_button(self) -> Button:
        return WindowsButton()
    
    def create_checkbox(self) -> Checkbox:
        return WindowsCheckbox()

class MacFactory(GUIFactory):
    def create_button(self) -> Button:
        return MacButton()
    
    def create_checkbox(self) -> Checkbox:
        return MacCheckbox()

# Usage
def create_gui(factory: GUIFactory):
    button = factory.create_button()
    checkbox = factory.create_checkbox()
    return button.paint(), checkbox.paint()
```

## 4. Builder Pattern
Separates the construction of a complex object from its representation.

```python
from dataclasses import dataclass
from typing import Optional

@dataclass
class Computer:
    cpu: str
    ram: str
    storage: str
    gpu: Optional[str] = None
    wifi: Optional[str] = None

class ComputerBuilder:
    def __init__(self):
        self.reset()
    
    def reset(self):
        self._computer = Computer(
            cpu="",
            ram="",
            storage=""
        )
    
    @property
    def computer(self):
        computer = self._computer
        self.reset()
        return computer
    
    def set_cpu(self, cpu: str):
        self._computer.cpu = cpu
        return self
    
    def set_ram(self, ram: str):
        self._computer.ram = ram
        return self
    
    def set_storage(self, storage: str):
        self._computer.storage = storage
        return self
    
    def set_gpu(self, gpu: str):
        self._computer.gpu = gpu
        return self
    
    def set_wifi(self, wifi: str):
        self._computer.wifi = wifi
        return self

# Usage
builder = ComputerBuilder()
computer = (builder
    .set_cpu("Intel i7")
    .set_ram("16GB")
    .set_storage("512GB SSD")
    .set_gpu("RTX 3080")
    .computer)
```

## 5. Prototype Pattern
Creates new objects by cloning an existing object, known as the prototype.

```python
from copy import deepcopy
from typing import Dict

class Prototype:
    def clone(self):
        return deepcopy(self)

class Document(Prototype):
    def __init__(self, name: str, content: Dict):
        self.name = name
        self.content = content
    
    def __str__(self):
        return f"Document: {self.name}, Content: {self.content}"

# Usage
original = Document("Original", {"section1": "Content 1", "section2": "Content 2"})
copy = original.clone()
copy.name = "Copy"
copy.content["section1"] = "Modified Content 1"

print(original)  # Original content remains unchanged
print(copy)      # Modified copy
```

## 6. Pool Pattern
Uses a set of initialized objects kept ready to use, rather than allocating and destroying them on demand.

```python
class DatabaseConnection:
    def __init__(self, connection_string: str):
        self.connection_string = connection_string
        self.is_connected = False
    
    def connect(self):
        self.is_connected = True
        print(f"Connected to {self.connection_string}")
    
    def disconnect(self):
        self.is_connected = False
        print("Disconnected")

class ConnectionPool:
    def __init__(self, connection_string: str, pool_size: int):
        self.connection_string = connection_string
        self._pool = []
        self._used = set()
        
        # Initialize pool
        for _ in range(pool_size):
            conn = DatabaseConnection(connection_string)
            self._pool.append(conn)
    
    def acquire(self):
        if not self._pool:
            raise Exception("No available connections")
        
        conn = self._pool.pop()
        self._used.add(conn)
        conn.connect()
        return conn
    
    def release(self, conn: DatabaseConnection):
        conn.disconnect()
        self._used.remove(conn)
        self._pool.append(conn)

# Usage
pool = ConnectionPool("postgresql://localhost:5432/db", 2)
conn1 = pool.acquire()
conn2 = pool.acquire()
pool.release(conn1)
conn3 = pool.acquire()  # Reuses the released connection
```

## Best Practices

1. **Choose the Right Pattern:**
   - Singleton for global state management
   - Factory Method for delegating object creation
   - Abstract Factory for families of related objects
   - Builder for complex object construction
   - Prototype for object cloning
   - Pool for resource management

2. **Consider Thread Safety:**
   ```python
   from threading import Lock
   
   class ThreadSafePool:
       def __init__(self):
           self._lock = Lock()
           self._resources = []
       
       def acquire(self):
           with self._lock:
               return self._resources.pop()
       
       def release(self, resource):
           with self._lock:
               self._resources.append(resource)
   ```

3. **Use Type Hints:**
   ```python
   from typing import List, Optional, TypeVar, Generic

   T = TypeVar('T')

   class GenericPool(Generic[T]):
       def __init__(self, items: List[T]):
           self._items: List[T] = items
       
       def acquire(self) -> Optional[T]:
           return self._items.pop() if self._items else None
   ```

## Common Pitfalls

1. **Overusing Singletons:**
   ```python
   # Bad: Global state can make testing difficult
   class GlobalConfig:
       _instance = None
       
       @classmethod
       def get_instance(cls):
           if cls._instance is None:
               cls._instance = cls()
           return cls._instance
   
   # Better: Dependency injection
   class Config:
       def __init__(self, settings: dict):
           self.settings = settings
   
   def create_app(config: Config):
       app = App()
       app.config = config
       return app
   ```

2. **Complex Factory Hierarchies:**
   ```python
   # Bad: Deep inheritance hierarchies
   class AbstractFactory:
       pass
   
   class ConcreteFactory1(AbstractFactory):
       pass
   
   class ConcreteFactory1A(ConcreteFactory1):
       pass
   
   # Better: Composition over inheritance
   class Factory:
       def __init__(self, creator_func):
           self.creator = creator_func
       
       def create(self):
           return self.creator()
   ```

3. **Ignoring Resource Cleanup:**
   ```python
   # Bad: Resource leak
   pool = ConnectionPool()
   conn = pool.acquire()
   # conn never released
   
   # Better: Context manager
   class ConnectionPool:
       def __enter__(self):
           self.conn = self.acquire()
           return self.conn
       
       def __exit__(self, exc_type, exc_val, exc_tb):
           self.release(self.conn)
   
   with ConnectionPool() as conn:
       # Use connection
       pass  # Auto-released
   ```

---


